In [1]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from imblearn.metrics import specificity_score
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import warnings
import os
os.environ["WANDB_MODE"] = "disabled"

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../dataset/CPPSet1.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=1024, stratify=df['Label'])
valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=1024, stratify=test_df['Label'])

train_df.to_csv('../dataset/CPPSet1-train.csv', index=False)
valid_df.to_csv('../dataset/CPPSet1-valid.csv', index=False)
test_df.to_csv('../dataset/CPPSet1-test.csv', index=False)

print("Done!")


Done!


In [3]:
model_name = '../Rostlab/prot_bert_bfd'

In [4]:
class DeepLocDataset(Dataset):
    def __init__(self, split="train", tokenizer_name=model_name, max_length=80):
        self.datasetFolderPath = '../dataset/'
        self.trainFilePath = os.path.join(self.datasetFolderPath, 'CPPSet1-train.csv')
        self.validFilePath = os.path.join(self.datasetFolderPath, 'CPPSet1-valid.csv')
        self.testFilePath = os.path.join(self.datasetFolderPath, 'CPPSet1-test.csv')
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        if split=="train":
            self.seqs, self.labels = self.load_dataset(self.trainFilePath)
        elif split=="valid":
            self.seqs, self.labels = self.load_dataset(self.validFilePath)
        else:
            self.seqs, self.labels = self.load_dataset(self.testFilePath)
        self.max_length = max_length
        
    def load_dataset(self,path):
        df = pd.read_csv(path,names=['input','labels'],skiprows=1)
        seq = list(df['input'])
        seq = [' '.join(i) for i in seq]
        label = list(df['labels'])
        assert len(seq) == len(label)
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

In [5]:
train_dataset = DeepLocDataset(split="train", tokenizer_name=model_name, max_length=70)
valid_dataset = DeepLocDataset(split="valid", tokenizer_name=model_name, max_length=70)
test_dataset = DeepLocDataset(split="test", tokenizer_name=model_name, max_length=70)

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    spec = specificity_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'specificity': spec,
        'f1': f1,
        'mcc' : matthews_corrcoef(labels, preds)
    }

In [7]:
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(model_name)#.cuda()
    #for param in model.parameters(): param.data = param.data.contiguous()
    return model

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    output_dir='../results',
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=1,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=30,
    fp16=True,
    fp16_opt_level="02",
    run_name="ProBert-BFD-MS",
    seed=3407)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics = compute_metrics)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Rostlab/prot_bert_bfd and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Rostlab/prot_bert_bfd and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate(valid_dataset)

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.save_model('../model/ProtBert_BFD_CPPSet1.pt')

In [ ]:
result = trainer.predict(test_dataset).predictions
golden = trainer.predict(test_dataset).label_ids
result = torch.argmax(torch.tensor(result), -1)

In [ ]:
result

In [ ]:
# torch.save(result, '../model/CPPSet1_result2.pt')